# BoxBunny Deployment & Operation
Use this notebook to build, calibrate, and run the boxing robot system.

**Workflow:**
1. **Setup**: Configure environment.
2. **Build**: Compile ROS workspace (if code changed).
3. **Calibrate**: Tune IMU for punches.
4. **Launch**: Run the Action Recognition System.


## 1. Environment Setup
Generates the environment file used by all subsequent commands.

In [ ]:
%%bash
cat > /tmp/boxbunny_env.sh <<'SH'
source ~/miniconda3/etc/profile.d/conda.sh
conda activate boxing_ai
export PYTHONNOUSERSITE=1
export SETUPTOOLS_USE_DISTUTILS=stdlib
source /opt/ros/humble/setup.bash
source /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/install/setup.bash 2>/dev/null || true
SH
echo "Environment ready at /tmp/boxbunny_env.sh"

## 2. Build Workspace
Run this if you have modified any C++ or ROS nodes.

In [ ]:
!bash -lc 'source /tmp/boxbunny_env.sh && \
  cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws && \
  colcon build --symlink-install'

## 3. Calibrate IMU
Launch this GUI to calibrate the punch thresholds. **Close it when done.**

In [ ]:
!bash -lc 'source /tmp/boxbunny_env.sh && ros2 run boxbunny_imu imu_punch_gui'

## 4. Run Action Recognition (Hybrid Mode)
Launches the main GUI with **YOLO Nano (Pose)** and **IMU Fusion**.
Click **[Calibrate IMU]** in the GUI if you haven't done step 3.

In [ ]:
!bash -lc 'source /tmp/boxbunny_env.sh && \
  cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/action_prediction && \
  python tools/inference/live_infer_rgbd.py \
    --model-config configs/rgbd_boxing_anticipation.py \
    --model-checkpoint /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/models/action_prediction_model/best_acc_82.4_epoch_161.pth \
    --yolo-model checkpoints/yolo26n.pt \
    --fps 30 \
    --rgb-res 640x480 \
    --depth-res 640x480'

## 5. Full ROS System (Deployment)
Launch the entire robot stack (LLM, Vision, Drill Manager, etc).

In [ ]:
!bash -lc 'source /tmp/boxbunny_env.sh && \
  ros2 launch boxbunny_bringup boxbunny_system.launch.py \
  enable_imu:=true enable_llm:=true enable_gui:=true'